Для начала установим библиотеку transformers. 
Более подробно про неё - https://huggingface.co/docs/transformers/index.
Если говорить кратко, то это библиотека от стартапа Hugging Face для работы с transformer-based моделями. (Для справки, transformer-based модели это такие модели, которые используют механизм внимания для повышения скорости обучения). 


In [ ]:
# pip install transformers

Для написания нейронной сети будем работать с русскоязычной моделью ruGPT3 от Сбера.
Немного информации про эту модель: (взято отсюда: https://developers.sber.ru/portal/products/rugpt-3?attempt=1)

RuGPT-3 умеет писать тексты на русском языке: например, может продолжить за вас историю, написать отзыв о продукте или твит.

Нейросеть обучена на русскоязычных текстах разных стилей: на энциклопедиях, социальных сетях, художественной и бизнес-литературе. С помощью такой модели можно выстроить с нуля даже рекомендательную систему.

In [1]:
#Импортируем необходимые библлиотеки
from transformers import (
    GPT2LMHeadModel, # The GPT2 Model transformer with a language modeling head on top (linear layer with weights tied to the input embeddings).
    GPT2Tokenizer, # GPT-2 BPE tokenizer, using byte-level Byte-Pair-Encoding.
    TextDataset, # Библиотеки, содержащие инструменты для подготовки датасета и даталоудера
    DataCollatorForLanguageModeling, 
    Trainer, # Библиотеки, необходимые для тренировки модели
    TrainingArguments
)

# torch - фреймворк машинного обучения для языка Python с открытым исходным кодом, созданный на базе. 
# Используется для решения различных задач: компьютерное зрение, обработка естественного языка.
import torch


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu') # Для выполнения всех вычислительных процессов на
# графическом процессоре

Для файнтюнинга скачаем модель поменьше, чтобы она точно поместилась на GPU.

In [2]:
# Загрузка, инициализация модели и токенизатора
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2" # small - модель поменьше
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

Теперь подготовим данные для тренировки. Датасет взят со следующего репозитория: https://github.com/dkagramanyan/astro/blob/main/horoscopes.csv.
Сразу оговорим, что датасет - огромный (содержит около 31000 гороскопов, а один гороскоп может содержать до 300 символов). Мы возьмем поменьше, первые 3600 гороскопов. Кроме этого нам надо будет убрать дату из этого датасета, а латинское название знака зодиака заменить на русское, чтобы он получился чистый и готовый к работе с нашей моделью. Воспользуемся обычными средствами Python.

In [3]:
# Предобработка датасета
f = open('horoscopes.txt', 'r', encoding='utf8')  # Открываем оригинальный датасет для чтения и записи
g = open('final_horoscopes.txt', 'w', encoding='utf8')  # Создаем файл для записи нашего датасета

signs = ['aries,"', 'gemini,"', 'taurus,"', 'cancer,"', 'leo,"', 'virgo,"', 'libra,"', 'scorpio,"', 'sagittarius,"',
         'capricorn,"', 'aquarius,"', 'pisces,"']  # То, что нам нужно заменить
signs_zamena = ['Овен: ', 'Близнецы: ', 'Телец: ', 'Рак: ', 'Лев: ', 'Дева: ', 'Весы: ', 'Скорпион: ', 'Стрелец: ',
                'Козерог: ', 'Водолей: ', 'Рыбы: '] # То, на что мы заменяем

lines = f.readlines()  # Считываем все строки из оригинального датасета
f.close()  # И сразу не забываем закрыть файл

i = 0  # Счетчик для отсчета 3000 гороскопов.

# Циклом будет редактировать строки и записывать их в новый файл
for line in lines:
    s = line.strip()
    s = s[11:]  # Удаляем дату
    for k in range(12):
        if signs[k] in s:
            s = s.replace(signs[k], signs_zamena[k])  # Заменяем латинское название знака зодиака на русское
    s = s[:len(s) - 1]
    s = s + '\n'
    g.write(s)  # Записываем в файл
    i += 1
    if i == 3600:
        break

g.close()  # После окончания работы - закрываем файл

Теперь подготовим обучающие данные для тренировки. Для этого нам потребуются два прекрасных готовых инструемнта, которые содержаться в библиотеке Transformers, необходимые для подготовки датасета и даталоудера. Для этого нам лишь потребуется один .txt файл с обучающими данными.

Этот блок для проверки работы модели.
Я просто взял небольшой кусок датасета и на нем обучил модель, чтобы посмотреть, как это будет работать

In [22]:
text = """Овен: Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем.
Близнецы: Первый день нового года прекрасно подходит для начинаний, которые вы постоянно откладывали в прошлом году. Отбросьте все сомнения и действуйте. Упор может быть сделан как на повышение профессиональной квалификации, так и на реформацию личной жизни.
Телец: Этот день как нельзя лучше подходит для улучшения состояния здоровья и пополнения физического потенциала. Посетите тренажерный зал, бассейн или начните делать зарядку по утрам.
Рак: Первый день нового года, да еще понедельник будет для вас спокойным, днем, когда вы будете сторонним наблюдателем всех событий, происходящих вокруг. Вам не придется лично принимать важные решения.
Лев: Львы не сорвутся в начале года с места в карьер, они немного устали. Первый день нового года ничем не примечателен и пройдет достаточно спокойно. Не переедайте и будьте умереннее в употреблении спиртных напитков.
Дева: Сегодняшнее расположение звезд может усилить дух соперничества, и вам лучше заняться чем-то более полезным, нежели личными предубеждениями, сплетнями и слухами. Поскольку месяц начался, следует мостить пути для более осмысленных дискуссий.
Весы: В этот день вы, судя по всему, будете в более благодушном настроении, чем в последнее время, и не станете придираться к другим. Это особенно кстати, если вы проводите время с любимым человеком. Есть вероятность, что вы будете заняты каким-то совместным проектом, который, скорее всего, принесет вам удовлетворение.
Скорпион: Этот день благоприятен для деловых встреч, знакомств, планирования деятельности на целый год. Надейтесь только на себя. Прирожденное чувство уверенности в себе и способность стойко переносить все испытания будут способствовать успеху.
Стрелец: Вероятны ухудшение самочувствия или проблемы с имуществом. Возможно, у вас появится желание что-либо изменить в жизни или отправиться в путешествие. Но не торопите события - сегодня не самый благоприятный день для долгосрочного планирования.
Козерог: Первый день года будет для вас днем, когда вам придется активно участвовать во всех событиях, происходящих вокруг, и лично принимать важные решения. В поступках и речах окружающих могут присутствовать мотивы, нарушающие ваше благополучие. Могут появиться тревоги и опасения.
Водолей: Удача на вашей стороне. Звезды предполагают неожиданный поворот в вашей жизни. Интуиция и предусмотрительность помогут вам найти правильное направление жизнедеятельности. Не исключен риск и смелые решения, которые могут принести немалую выгоду. Главное - вовремя остановиться.
Рыбы: Вы чувствуете себя слегка усталым и склоняетесь к мысли о том, что ваша жизнь слишком упорядочена и стабильна. Вам хочется приключений и встреч с новыми интересными людьми. Вас будут отговаривать, но вы все равно поступите по-своему.
Овен: Обычно вы хорошо организованный человек, но временами попадаете в затруднительное положение. Сегодняшнее расположение планет намекает именно на это. Закончите все, что начали, не ешьте сегодня что попало некоторые могут отравиться, возможно, рыбой.
Близнецы: Сегодня вы проявите активность в общественной деятельности, среди большого скопления людей. Скорее всего это будет тем или иным образом связано с работой и вашими профессиональными обязанностями. В отдельных случаях - с вашими личными планами и желаниями.
Телец: Соблюдайте осторожность в ситуациях, которые могут оказаться аварийными. У вас подъем творческих сил, используйте их так, чтобы осталось время на чтение. Не пугайтесь сложной литературы, вы ее усвоите.
Рак: Сегодня подходящий день для того, чтобы насладиться приятной компанией друзей и соратников. Вы будете в центре внимания у своих слушателей, кем бы они ни были - соседями по лестничной площадке, зрителями в зале или клиентами в фирме.
Лев: Некоторые Львы весьма удивят окружающих людей странностями своего поведения и неожиданными поступками, никак не сочетающимися с привычным для всех обликом и стилем их жизни. Такова их натура - дня не могут прожить, чтобы не удивить кого-то. Не исключено обострение хронических недугов, особенно если вы по невнимательности перегрузитесь.
Дева: Нет резона пытаться в чем-то убедить других или даже просить их об одолжении. Они слишком подавлены, чтобы уделить вам внимание, но вы не должны принимать это на свой счет, поскольку данная фаза продлится лишь сутки. Однако как человек здравомыслящий, вы, несомненно, осознаете свою ошибку и вскоре поторопитесь к выходу.
Весы: Другие будут либо обманывать самих себя, либо пытаться обмануть вас. Теперь, когда вас предупредили, вы будете начеку и станете относиться к тому, что они скажут вам, с меньшим доверием. Кроме того, если вы страдаете какой-либо пищевой аллергией, проявляйте особую осмотрительность в отношении еды, иначе вы рискуете оказаться в постели еще до конца дня.
Скорпион: Сегодня возможно, что у вас возникнет острое чувство неудовлетворенности. Личные отношения в начале года будут складываться не так, как вам хотелось бы.
Стрелец: Не рекомендуется заниматься делами, связанными с риском, азартными играми. Благоприятен отдых в кругу семьи, игры, прогулки в одиночестве. Во второй половине дня приготовьтесь услышать неожиданную приятную новость.
Козерог: Этот день как нельзя лучше подходит для улучшения состояния здоровья и пополнения физического потенциала. Посетите сегодня тренажерный зал, бассейн или просто начните делать зарядку по утрам.
Водолей: День самозащиты, отстаивания своих принципов. В этот день влияние некоторых планет на вашу жизнь заметно ослабеет. Проявив свойственную вам способность к анализу, мягкость и скромность, вы сможете изменить ситуацию к лучшему.
Рыбы: К сожалению, после вчерашнего многообещающего старта, звезды несколько изменили свое положение. Это означает либо пустую трату денег, либо то, что кто-то пытается обманом выманить у вас кругленькую сумму. Вам не помешает быть менее доверчивым и более осторожным. Советуем не доверять даже собственным глазам.

"""

In [23]:

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding='utf8') as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Loading features from cached file cached_lm_GPT2Tokenizer_64_train_dataset.txt [took 0.004 s]


А этот блок ведет уже к готовым обучающим данным, которые получились в результате преодобработки оригинального датасета.

In [4]:
# Укажем путь к обучающим данным
train_path = 'final_horoscopes.txt'

# Создание датасета
Train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
# Создание даталодера (нарезает текст на оптимальные по длине куски)
Data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

C:\Users\Никита\AppData\Roaming\Python\Python39\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Теперь, для файнтюнинга (В отличие от feature extraction, суть дообучения (Fine Tuning) заключается в размораживании последних слоев нейронной сети (Neural Net) и их обучении. Таким образом, корректируются слои, которые имеют наиболее абстрактные представления. Производя дообучение только нескольких слоев, мы уменьшаем риск переобучения (overfitting). И самое главное, это позволяет сделать текущую модель ещё более подходящей к нашей задаче) нам понадобиться объект класса Trainer, который сделает всю необходимю работу вместо нас и далее просто нужно будет запустить trainer.train().

In [5]:
batch_size = 16
num_epochs = 200

training_args = TrainingArguments(
    output_dir="./finetuned_final", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=num_epochs, # number of training epochs
    per_device_train_batch_size=batch_size, # batch size for training
    # The batch size defines the number of samples that will be propagated through the network.
    per_device_eval_batch_size=batch_size,  # batch size for evaluation
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=16, # to make "virtual" batch size larger
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=Data_collator,
    train_dataset=Train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

Запускаем тренировку!

In [6]:
# У меня возникала ошибка CUDA out of memory и пришлось прибегнуть к следующим функциям, чтобы очистить кэш CUDA и 
# функции сборщика мусора gc.collect(). Как ни странно, но именно после комбинации этих двух методов у меня
# всё заработало и модель начала спокойно тренироваться.

import gc
gc.collect()

torch.cuda.empty_cache()

In [7]:
trainer.train()

***** Running training *****
  Num examples = 4179
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 3200
  Number of trainable parameters = 125231616


  0%|          | 0/3200 [00:00<?, ?it/s]

Saving model checkpoint to ./finetuned_final\checkpoint-500
Configuration saved in ./finetuned_final\checkpoint-500\config.json


{'loss': 2.4249, 'learning_rate': 8.463949843260189e-06, 'epoch': 31.24}


Model weights saved in ./finetuned_final\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./finetuned_final\checkpoint-1000
Configuration saved in ./finetuned_final\checkpoint-1000\config.json


...

Model weights saved in ./finetuned_final\checkpoint-2500\pytorch_model.bin
Saving model checkpoint to ./finetuned_final\checkpoint-3000
Configuration saved in ./finetuned_final\checkpoint-3000\config.json


{'loss': 0.9027, 'learning_rate': 6.269592476489028e-07, 'epoch': 187.49}


Model weights saved in ./finetuned_final\checkpoint-3000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 14057.3868, 'train_samples_per_second': 59.456, 'train_steps_per_second': 0.228, 'train_loss': 1.339286332130432, 'epoch': 199.98}


TrainOutput(global_step=3200, training_loss=1.339286332130432, metrics={'train_runtime': 14057.3868, 'train_samples_per_second': 59.456, 'train_steps_per_second': 0.228, 'train_loss': 1.339286332130432, 'epoch': 199.98})

Сохраняем и загружаем дообученную модель.

Эта папка храниться по следующей ссылке: https://drive.google.com/drive/folders/1sdUoxAginZBniiP1RLdd-PXTjvNVExoW?usp=sharing

In [8]:
output_dir = 'horoscopes_rugpt3_final' # Директория, где лежит наша финальная модель

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Configuration saved in horoscopes_rugpt3_final\config.json
Model weights saved in horoscopes_rugpt3_final\pytorch_model.bin
tokenizer config file saved in horoscopes_rugpt3_final\tokenizer_config.json
Special tokens file saved in horoscopes_rugpt3_final\special_tokens_map.json


('horoscopes_rugpt3_final\\tokenizer_config.json',
 'horoscopes_rugpt3_final\\special_tokens_map.json',
 'horoscopes_rugpt3_final\\vocab.json',
 'horoscopes_rugpt3_final\\merges.txt',
 'horoscopes_rugpt3_final\\added_tokens.json')

Загрузка уже обученной модели

In [2]:
# Это используется , если нужно загрузить уже готовую, обученную модель.
from transformers import AutoConfig

# AutoConfig is a generic configuration class that will be instantiated as one of 
# the configuration classes of the library when created with the from_pretrained() class method.

config = AutoConfig.from_pretrained("./horoscopes_rugpt3_final")

from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("./horoscopes_rugpt3_final")

# Загружаем обученную модель из папки, куда мы ее сохраняли
model = GPT2LMHeadModel.from_pretrained("./horoscopes_rugpt3_final",config=config).to(DEVICE) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Пример с генерацией текста. При генерации текста можно будет играться со следующими параметрами:
1) temperature — параметр сглаживания; чем выше, тем сильнее сглаживание вероятностного распределения токенов при предсказании;
2) top_p — техника сэмплирования: сортировка предсказаний каждого следующего слова по вероятностям и отсекание вариантов, как только суммарная вероятность предыдущих токенов превысит p;
3) max_length — максимальная длина генерируемого текста;

In [3]:
#Пример вероятностного сэмплирования (При такой генерации мы берём не самый вероятный токен, 
# а выбираем его "случайно" с учётом распределения вероятностей. ) с некоторым ограничением.

text = "Водолей: \n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval() # model.eval() is a kind of switch for some specific layers/parts of the model that 
#behave differently during training and inference (evaluating) time.

# Описание torch.no_grad() на английском:
# With torch.no_grad() method is like a loop in which 
# every tensor in that loop will have a requires_grad set to False. 
# It means that the tensors with gradients currently attached to the current computational 
# graph are now detached from the current graph and no longer we will be able to compute the gradients 
# with respect to that tensor. Until the tensor is within the loop it is detached from the current graph. 
# As soon as the tensor defined with gradient is out of the loop, it is again attached to 
# the current graph. This method disables the gradient calculation which reduces the memory 
# consumption for computations.

with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100,
                        )

# Здесь происходит декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Водолей: 
 ╦Удача сопутствует вам во всех начинаниях, особенно в исследовательской работе, поиске более совершенных методов и технологий, в экспериментах и поиске более совершенных видов деятельности. Хороший день для научных исследований, для прогрессивных технологических или методологических изменений на производстве и в бизнесе. Вы можете получить финансовую помощь для реализации новых проектов. Возросшее обаяние и энергичность Водолеев позволит им преодолеть все преграды и препятствия на пути к успеху. Однако не забывайте, что ""


В следующем блоке приведу гороскопы, сгенерированные для каждого из 12 знаков зодиака.

In [20]:
Oven = '''Овен: Сегодня вероятны конфликты на работе из-за идей и мнений 
          партнеров по работе, особенно касающихся совместного капитала, акционирования, 
          налогов и пошлин, алиментов и наследства, долговых обязательств. Возможны столкновения с законом в той или иной форме. 
          Вы склонны проявлять честность и прямоту во что бы то ни стало. Общение динамично и конструктивно, особенно с женщинами - партнерами и 
          сотрудницами. Возможны взаимное пренебрежение интересами партнеров и коллег. Совместный'''

Bliznetsi = '''Близнецы: День обещает быть насыщенным интересными событиями в личной жизни. Звезды 
               рекомендуют избегать коллективной работы. Вероятны неожиданные препятствия или недоразумения, 
               связанные с общением, общением или личными отношениями. Не исключено, что у вас появятся новые интересные предложения, 
               однако не торопите события - они могут затянуться надолго. День обещает быть насыщенным интересными событиями в 
               личной жизни. Звезды рекомендуют избегать коллективной работы. Вероятны неожиданные препятствия или недоразумения, 
               связанные с общением'''

Telets = '''Телец: Вы настроены оптимистично, имеете тенденцию к деловой и финансовой экспансии. Благоприятны контакты с 
            различными группами и организациями, привлечение дружеских связей, но стремление к лидерству и игнорирование чужих 
            интересов может привести к конфликтам с друзьями, отказу в помощи. Возможны заблуждения относительно перспектив и 
            реалистичности проектов. Активизируется деловое партнерство, укрепляются связи. Хороший день для заключения сделок, 
            открытия нового предприятия или реорганизации производства. Вы ощутите возрастание активности'''

Rak = '''Рак: Жизненный потенциал имеет тенденцию к снижению. Жизненный потенциал может несколько снизиться. 
         Вероятны осложнения в отношениях с детьми. Не затягивайте с их рождением.
         Неожиданные недоразумения, возможен крах карьеры, неприятные изменения на работе. Вы стремитесь к свободе 
         действий любой ценой, но отрицаете ответственность за свои поступки. Из-за эгоизма и упрямства можете 
         пренебречь многими хорошими советами. Поддерживаете сумасбродные идеи и придерживаетесь ненадежных перспектив.'''

Lev = '''Лев: Займись саморекламой, общением с важными людьми, подведи черту под старыми планами, разбери старые связи. 
         Сегодня не самый подходящий день для того, чтобы заниматься саморекламой. (p.s. Немножко противоречиво получилось -_-)
         Неожиданные события во многих сферах жизни ослабляют возможность коммерческого, финансового успеха. Вы можете ощущать 
         некоторую незащищенность и безотчетный страх, что скажется на Ваших отношениях с сотрудниками или компаньонами. 
         Неожиданные происшествия и разочарования в работе, карьере.'''

Deva = '''Дева: Удача на вашей стороне, а непоколебимость жизненных принципов может положительно повлиять на ваши успехи. 
          Хороший день для активного решения общих с партнерами или сотрудниками вопросов, для открытия новых предприятий 
          и начала проектов, для командировок и ответственных визитов. Вам поможет дипломатичность, такт, хороший вкус и манеры. 
          Вы сможете кого угодно убедить в чем угодно, а потому можете не сомневаться в искренности партнеров и в лояльности начальства.
          День характеризуется умением находить практическое применение своему вдохновению'''

Vesi = '''Весы: Близкие вам люди будут страдать, если вы не выложите все свои карты. 
          Не реагируйте слишком болезненно на то, что происходит, потому что вы - хозяин положения. 
          Будьте терпеливы, и все уладится. Возможно, вам придется взять на себя лидерство в решении некоторых вопросов, 
          особенно касающихся образования и культурных мероприятий. Взаимопонимание на работе улучшается. 
          Возможны новые контакты, которые повлияют на дальнейшую судьбу людей, с которыми вам предстоит общаться.'''

Scorpion = '''Скорпион: Вы можете полностью проявить все свои интеллектуальные и духовные способности, 
              однако не торопитесь с выводами, подождите немного - может быть, они будут для Вас неожиданными. 
              Не торопитесь с выводами. Вы можете полностью проявить все свои интеллектуальные и духовные способности, 
              однако не торопитесь с выводами, подождете немного - может быть, они будут для Вас неожиданными. 
              Не торопитесь с выводами. Вы можете полностью проявить все свои интеллектуальные и духовные способности, однако'''  

Strelets = '''Стрелец: В вашей жизни намечается виток на иной уровень развития, к новым делам или новым делам, 
              в которых вы давно уже не были. Сегодня звезды рекомендуют проявлять осмотрительность и сдержанность в общении. 
              Постарайтесь не проявлять агрессию по отношению к окружающим людям. День несет двойственную 
              характеристику. Первая половина дня благоприятна для путешествий. Вторая половина дня - неблагоприятна 
              для активных физических нагрузок. В первой половине дня вероятны ссоры или травмы. 
              Нельзя расслабляться, отказывать'''

Kozerog = '''Козерог: Благоприятный день для налаживания отношений с женщинами - партнерами и сотрудницами. 
             Благоприятный день для обращения к богатым и влиятельным женщинам. Ваша активность на работе 
             будет способствовать успеху. Вы сможете максимально проявить свои интеллектуальные и духовные способности, даже будучи в депрессии.
             Если сегодня заболеете, то болезнь будет долгой, тяжелой и тяжелой, возможны осложнения. 
             Будьте осторожны при управлении транспортом и работе с острыми предметами, техникой.
             Благоприятный день для налаживания отношений'''    

Vodolei = '''Водолей: Водолеям следует определить границы своих возможностей. Водолеям следует обратить внимание 
             на качество питания. Звезды предостерегают от слишком бурного проявления эмоций критика по отношению к окружающим 
             может обернуться против вас. Неблагоприятный день для общения с друзьями и родственниками. Звезды предостерегают от 
             активных физических нагрузок и коллективных мероприятий. Звезды предостерегают Водолеев от активных 
             физических нагрузок и коллективных мероприятий. Звезды предостерегают от'''

Ribi = '''Рыбы: День благоприятен для долгосрочного планирования, заключения соглашений, 
          ответственных выступлений, заключения сделок, для торговых дел, особенно касающихся недвижимости.  
          В то же время возможны некоторые упущения в работе.  Активизируются предубеждения и предрассудки. 
          Вы склонны к мечтательности, отрыву от реальности.  Неблагоприятны сделки и финансовые операции, 
          особенно связанные с недвижимостью.  Покупки окажутся неудачными, а траты чрезмерными. Возможны ощутимые убытки. 
          Возмож'''

Список использованных источников:

Использованные источники:

1. https://www.geeksforgeeks.org/what-is-with-torch-no_grad-in-pytorch/
2. https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
3. https://habr.com/ru/post/704592/
4. https://huggingface.co/transformers/v3.0.2/index.html
5. https://habr.com/ru/post/417215/
6. https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network
7. https://python-school.ru/blog/fine-tuning/
8. https://habr.com/ru/post/334380/
9. https://developers.sber.ru/portal/products/rugpt-3?attempt=1
10. https://habr.com/ru/post/486358/
11. https://github.com/lambdamai/datascience/blob/main/nlp/GPT_introduction.ipynb (САМОЕ ГЛАВНОЕ!!! СПАСИБО ОГРОМНОЕ ЗА ГАЙД!!!)